In [148]:
import cv2
import math
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort

In [149]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

In [150]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 2050


In [151]:
model_path = r"C:\Users\Rishabh Surana\Desktop\ATMS project\Custom model for Object Detection\CNN_obj_Traffic_detector_4.pth"
video_path = r"C:\Users\Rishabh Surana\Desktop\ATMS project\test_data\test_video\video 1.avi"

In [152]:
def get_model(num_classes):
    model = fasterrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model
num_classes = 22
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = get_model(num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()
    

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [153]:
tracker = DeepSort(max_age=30)

In [154]:
cap = cv2.VideoCapture(video_path)

In [155]:
if not cap.isOpened():
    print("Error : Could not open video")
    exit()
else:
    print("Video is opened and working")

Video is opened and working


In [156]:
def obj_Detection(results, detections):
    for box in results.boxes:
        x1, y1, x2, y2 = map(int,box.xyxy[0])
        class_id = int(box.cls[0])
        class_name = results.names[class_id]
        confidence = float(box.conf[0])
        
        detections.append(([x1,y1,x2-x1,y2-y1], confidence, class_name))

In [157]:
stall_cars = []
def stall_movement_fn(car_coordinates, car_id):
    def eucledian(base_coord, pt):
        pt1 = (base_coord[0] - pt[0])**2
        pt2 = (base_coord[1] - pt[1])**2
        dist = math.sqrt(pt1+pt2)
        return dist

    coords = car_coordinates[car_id]
    stalled = True
    if(len(coords) > 10):
        recent_coords = coords[-10:]
        base_coord = recent_coords[0]
        for pt in recent_coords[1:]:
            distance = eucledian(base_coord, pt)
            if distance >= 5: #pixel_threshold
                stalled = False
                break
    else:
        stalled = False
        
    return stalled
    
    

In [158]:
def alert(car_id, reason):
    if(reason=="Opposite_direction"):
        print(f"{car_id} is moving in Opposite Direction.")
    elif(reason=="Stalled"):
        print(f"{car_id} is stalled.")
    else:
        print(f"{car_id} is ok")

In [159]:
opposite_cars = []
def detect_car_movement(car_coordinates, expected = [0,-1], threshold = 0.4):
    
    def normalize_vector(v):
        mag = math.sqrt(v[0]**2 + v[1]**2)
        if(mag!=0):
            return [v[0]/mag, v[1]/mag]
        else:
            return [0,0]
        
    def cosine_similarity(movment_vec, expected):
        dot_num = (movment_vec[0] * expected[0]) + (movment_vec[1] * expected[1])
        mag1 = math.sqrt((movment_vec[0]**2)+(movment_vec[1]**2))
        mag2 = math.sqrt((expected[0]**2)+(expected[1]**2))
        if(mag1 and mag2):
            return dot_num / (mag1 * mag2)
        else:
            return 1
    
    for car_id, coords in car_coordinates.items():
        
        if(len(coords)<2):
            continue
        
        x1, y1 = coords[-2] 
        x2, y2 = coords[-1]
        
        if(x2-x1 == 0 and y2-y1 == 0):
            if(stall_movement_fn(car_coordinates, car_id)):
                if(car_id not in stall_cars):
                    stall_cars.append(car_id)
                    alert(car_id,"Stalled")
        movment = [x2-x1, y2-y1]
        movement_unit = normalize_vector(movment)
        similarity = cosine_similarity(movement_unit, expected)
        
        
        if(similarity < -threshold):
            if(car_id not in opposite_cars):
                alert(car_id,"Opposite_direction")
                opposite_cars.append(car_id)
        

In [160]:
car_coordinates = {}
def car_coords_fn(track_id, x1, y1, x2, y2):
    center_x = int((x1 + x2)/2)    
    center_y = int((y1 + y2)/2)
    if track_id not in car_coordinates:
        car_coordinates[track_id] = []
    if len(car_coordinates[track_id]) >= 20:
        car_coordinates[track_id].pop(0)
    car_coordinates[track_id].append([center_x,center_y])
    detect_car_movement(car_coordinates)

In [161]:
def draw_Tracks(tracks,frame):
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        class_name = track.get_det_class()
        
        car_coords_fn(track_id, x1, y1, x2, y2)
    
        label = f"ID {track_id} | {class_name}"
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(frame, label, (x1, y1 - 10,), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)
        

In [162]:
def predict_custom_model(model, frame, threshold = 0.5):
    image_resized = cv2.resize(frame, (512,512))
    image_tensor = F.to_tensor(image_resized).to(device)
    
    with torch.no_grad():
        outputs = model([image_tensor])[0]
        
    keep = outputs['scores'] > threshold
    boxes = outputs['boxes'][keep].cpu()
    labels=outputs['labels'][keep].cpu()
    scores=outputs['scores'][keep].cpu()
    
    detections = []
    scale_x = frame.shape[1]/512
    scale_y = frame.shape[0]/512
    
    for box, score, label in zip(boxes, scores, labels):
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = int(x1*scale_x), int(y1*scale_y), int(x2*scale_x), int(y2*scale_y)
        w,h = x2 - x1, y2 - y1
        detections.append(([x1,y1,w,h], float(score), label))
    return detections

In [163]:
while True:
   ret, frame = cap.read()
   if not ret:
       break
   detections = predict_custom_model(model, frame, threshold=0.6)
   tracks = tracker.update_tracks(detections, frame = frame)
   
   draw_Tracks(tracks,frame)
   
   cv2.imshow("Output", frame)
   if(cv2.waitKey(1) and 0xFF == ord('q')):
        break

cap.release()
cv2.destroyAllWindows()
   
   

7 is moving in Opposite Direction.
9 is moving in Opposite Direction.
7 is stalled.
9 is stalled.
3 is moving in Opposite Direction.


In [164]:
total_cars = len(car_coordinates)
print(f"Total Cars = {total_cars}")
print("--"*130)

for track_id, coords in car_coordinates.items():
    print(f"Car Id {track_id} -> {coords}")

Total Cars = 17
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Car Id 1 -> [[179, 125], [178, 123], [178, 122], [178, 121], [178, 119], [177, 118], [177, 117], [177, 116], [176, 115], [176, 114], [176, 113], [176, 112], [175, 111], [175, 110], [174, 109], [174, 107], [174, 106], [173, 105], [173, 104], [172, 103]]
Car Id 2 -> [[174, 99], [174, 98], [174, 97], [174, 96], [173, 95], [173, 93], [173, 92], [173, 91], [173, 90], [173, 89], [172, 88], [172, 87], [172, 86], [172, 84], [171, 83], [171, 82], [171, 81], [171, 80], [171, 79], [171, 78]]
Car Id 3 -> [[214, 139], [213, 137], [212, 135], [211, 134], [210, 132], [209, 130], [210, 131], [209, 130], [209, 129], [208, 127], [207, 126], [206, 125], [206, 124], [205, 123], [204, 121], [204, 121], [203, 120], [203, 118], [202, 11

In [165]:
print("\n---Stalled Vehicle---\n")
for s_car in stall_cars:
    print(f"Car Id {s_car} -> Stalled.")
print("\n\n---Opposite Vehicle---\n")
for op_car in opposite_cars:
    print(f"Car Id {op_car} -> Opposite direction.")


---Stalled Vehicle---

Car Id 7 -> Stalled.
Car Id 9 -> Stalled.


---Opposite Vehicle---

Car Id 7 -> Opposite direction.
Car Id 9 -> Opposite direction.
Car Id 3 -> Opposite direction.
